In [1]:
import sys
print(sys.executable)


C:\Users\KATANA\Desktop\Jupyter-Lab\workshop\slip_recognition\.env\Scripts\python.exe


In [2]:
import os
os.getcwd()

'C:\\Users\\KATANA\\Desktop\\Jupyter-Lab\\workshop\\slip_recognition\\notebook'

In [3]:
import cv2
import json
import numpy as np
import os
import paddle
from paddleocr import PaddleOCR

C:\Users\KATANA\Desktop\Jupyter-Lab\workshop\slip_recognition\.env\lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
C:\Users\KATANA\Desktop\Jupyter-Lab\workshop\slip_recognition\.env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Checking connectivity to the model hosters, this may take a while. To bypass this check, set `DISABLE_MODEL_SOURCE_CHECK` to `True`.


In [4]:
slip_template_image_path = r"../data/templates/slip_template.jpeg"

In [21]:
IMAGE_PATH = slip_template_image_path
OUTPUT_JSON = "../configs/roi_config.json"

def imread_unicode(path):
    data = np.fromfile(path, dtype=np.uint8)
    return cv2.imdecode(data, cv2.IMREAD_COLOR)

img = imread_unicode(IMAGE_PATH)
if img is None:
    raise FileNotFoundError("Cannot load image")

clone = img.copy()
drawing = False
start_point = None
rois = {}

def mouse_callback(event, x, y, flags, param):
    global drawing, start_point, img

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_point = (x, y)

    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        img = clone.copy()
        cv2.rectangle(img, start_point, (x, y), (0, 255, 0), 2)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        end_point = (x, y)
        cv2.rectangle(img, start_point, end_point, (0, 255, 0), 2)

        name = input("Enter ROI name (amount / fee / transaction_id): ").strip()
        if name:
            rois[name] = {
                "x1": min(start_point[0], end_point[0]),
                "y1": min(start_point[1], end_point[1]),
                "x2": max(start_point[0], end_point[0]),
                "y2": max(start_point[1], end_point[1])
            }
            print(f"✅ Saved ROI: {name}")

cv2.namedWindow("Select Multiple ROI", cv2.WINDOW_NORMAL)  # allow resizing
cv2.resizeWindow("Select Multiple ROI", 800, 800)          # set width x height
cv2.setMouseCallback("Select Multiple ROI", mouse_callback)            

cv2.namedWindow("Select Multiple ROI")
cv2.setMouseCallback("Select Multiple ROI", mouse_callback)

print("🖱️ Drag to select ROI")
print("⌨️ Type ROI name in terminal")
print("💾 Press S to save all | Q to quit")

while True:
    cv2.imshow("Select Multiple ROI", img)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("s"):
        with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
            json.dump(rois, f, indent=2, ensure_ascii=False)
        print(f"✅ All ROIs saved to {OUTPUT_JSON}")
        break

    elif key == ord("q"):
        break

cv2.destroyAllWindows()

🖱️ Drag to select ROI
⌨️ Type ROI name in terminal
💾 Press S to save all | Q to quit


Enter ROI name (amount / fee / transaction_id):  datetime_transaction


✅ Saved ROI: datetime_transaction


Enter ROI name (amount / fee / transaction_id):  account_name


✅ Saved ROI: account_name


...

✅ Saved ROI: transaction_id


Enter ROI name (amount / fee / transaction_id):  amount


✅ Saved ROI: amount


Enter ROI name (amount / fee / transaction_id):  fee


✅ Saved ROI: fee
✅ All ROIs saved to roi_config.json


In [22]:
ocr = PaddleOCR(
    lang="th",                                # Thai language model
    device="gpu",
    ocr_version="PP-OCRv5",                   # Latest OCR pipeline
    use_doc_orientation_classify=False,       # Skip document orientation (not needed for receipts)
    use_textline_orientation=False,           # Skip textline orientation (Thai is mostly horizontal)
    use_doc_unwarping=False,                  # Skip unwarping (receipt is already flat)
    text_det_limit_side_len=800,              # Resize long side to 960px for better detection
    text_det_limit_type="max",                # Limit resizing by max side
    text_det_thresh=0.3,                      # Lower threshold for small text boxes
    text_det_box_thresh=0.5,                  # Box confidence threshold
    text_det_unclip_ratio=2.0,                # Expand detected boxes slightly
    text_rec_score_thresh=0.5,                # Recognition confidence threshold
    return_word_box=True                      # Return word-level boxes
)


Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\KATANA\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('th_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\KATANA\.paddlex\official_models\th_PP-OCRv5_mobile_rec`.


In [17]:
print("Is GPU available?", paddle.is_compiled_with_cuda())
print("Current device:", paddle.get_device())

Is GPU available? True
Current device: gpu:0


# Sample

In [18]:
def imread_unicode(path):
    data = np.fromfile(path, dtype=np.uint8)
    return cv2.imdecode(data, cv2.IMREAD_COLOR)

directory = os.path.join(os.getcwd(), r"../data/raw")

# Loop through all files in the directory
for filename in os.listdir(directory):
    full_path = os.path.join(directory, filename)

    if os.path.isfile(full_path):
        _, extension = os.path.splitext(filename)
        if extension.lower() == '.jpeg':   # only process JPEGs
            img = imread_unicode(full_path)

            if img is None:
                print(f"❌ Could not load {filename}")
                continue

            # Run OCR on the entire image
            ocr_result = ocr.predict(img)

            # Collect recognized text
            results = [item["rec_texts"] for item in ocr_result]

            print(f"📄 {filename} → {results}")

📄 016006200912DPM08522.jpeg → [['เติมเงินสำเร็จ', '6', 'ม.ค.', '69', '9 20:09 น.', 'นาย', 'ธีรภัทร พ', 'ธ.กสิกรไทย', 'Xxx-x-x5498-x', '2026 PLAN', 'TrueMoney Wallet', 'truemoney', '0981702030', 'wallet', 'เลขที่รายการ:', '016006200912DPM08522', 'จำนวน:', '300.00 บาท', 'ค่าธรรมเนียม:', '0.00 บาท', 'สแกนตรวจสอบสลิป']]
📄 016008121452CPP06690.jpeg → [['โอนเงินสำเร็จ', '8O', '8 ม.ค.', '69', '12:14 น.', 'นาย', 'ธีรภัทร พ', 'ธ.กสิกรไทย', 'Xxx-x-x5498-x', '2026 PLAN', 'นายดนุสรณ์ ใจบุญ', 'Prompt', 'Pay', 'รหัสพร้อมเพย์', 'Q', 'xxx-xxx-4937', 'เลขที่รายการ:', '016008121452CPP06690', 'จำนวน:', '3,080.00 บาท', 'ค่าธรรมเนียม:', '0.00 บาท', 'สแกนตรวจสอบสลิป']]
📄 016008190840130455.jpeg → [['ชำระเงินสำเร็จ', '8 ม.ค.', '69', '19:08 น.', 'นาย', 'ธีรภัทร พ', 'ธ.กสิกรไทย', 'Xxx-x-x5498-x', 'ไลน์แมน', '2026 PLAN', '00420260108190808ZRJ', '00420260108190808ZRJ', 'เลขที่รายการ:', '016008190840130455', 'จำนวน:', '215.00 บาท', 'ค่าธรรมเนียม:', '0.00 บาท', 'สแกนตรวจสอบสลิป']]
📄 016009173621DPM08676.jpeg → [['

# Full Version

In [23]:
def imread_unicode(path):
    data = np.fromfile(path, dtype=np.uint8)
    return cv2.imdecode(data, cv2.IMREAD_COLOR)

# Reference image used when you first defined ROIs
reference_image_path = "../data/templates/slip_template.jpeg"
OUTPUT_JSON = "../configs/roi_config_normalized.json"

# Load reference image
img = cv2.imread(reference_image_path)
if img is None:
    raise FileNotFoundError("Cannot load reference image")

h, w = img.shape[:2]

# Load original ROI config (absolute pixel coordinates)
with open("../configs/roi_config.json", "r", encoding="utf-8") as f:
    rois = json.load(f)

# Convert to normalized ratios
normalized_rois = {}
for name, box in rois.items():
    normalized_rois[name] = {
        "x1": box["x1"] / w,
        "y1": box["y1"] / h,
        "x2": box["x2"] / w,
        "y2": box["y2"] / h,
    }

# Save normalized ROI config
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(normalized_rois, f, indent=2, ensure_ascii=False)

print(f"✅ Normalized ROI config saved to {OUTPUT_JSON}")


✅ Normalized ROI config saved to roi_config_normalized.json


In [28]:
def imread_unicode(path):
    data = np.fromfile(path, dtype=np.uint8)
    return cv2.imdecode(data, cv2.IMREAD_COLOR)

# Load normalized ROI config
with open("../configs/roi_config_normalized.json", "r", encoding="utf-8") as f:
    rois = json.load(f)

directory = os.path.join(os.getcwd(), r"../data/raw")

all_results = {}  # collect results for all files

# Loop through all JPEG files in the directory
for filename in os.listdir(directory):
    full_path = os.path.join(directory, filename)

    if os.path.isfile(full_path):
        _, extension = os.path.splitext(filename)
        if extension.lower() == ".jpeg":
            img = imread_unicode(full_path)
            if img is None:
                print(f"❌ Could not load {filename}")
                continue

            h, w = img.shape[:2]
            results = {}

            for name, box in rois.items():
                # Scale normalized ratios back to pixel coordinates
                x1 = int(box["x1"] * w)
                y1 = int(box["y1"] * h)
                x2 = int(box["x2"] * w)
                y2 = int(box["y2"] * h)

                crop = img[y1:y2, x1:x2]

                if crop.size == 0:
                    print(f"Skipping ROI '{name}' in {filename}, crop is empty")
                    continue

                # Run OCR safely
                ocr_result = ocr.predict(crop, return_word_box=False)
                if "rec_texts" in ocr_result[0]:
                    results[name] = ocr_result[0]["rec_texts"]
                    
            #print(f"📄 {filename} → {results}")
            # Store results for this file
            all_results[filename] = results

# Save all results into one JSON file
with open("../data/processed/ocr_results.json", "w", encoding="utf-8") as f:
    json.dump(all_results, f, indent=2, ensure_ascii=False)

print("")
print("✅ All OCR results saved to ocr_results.json")


✅ All OCR results saved to ocr_results.json


In [29]:
import pandas as pd
# Convert results to pandas DataFrame
df = pd.DataFrame.from_dict(all_results, orient="index")

# Save DataFrame to CSV (optional)
df.to_csv("../data/processed/ocr_results.csv", encoding="utf-8-sig")

print("✅ OCR results also saved to ocr_results.csv")
df


✅ OCR results also saved to ocr_results.csv


,datetime_transaction,account_name,destination account,account_number,transaction_id,amount,fee
004999086098141_20250101_030552.jpeg,[],[],[],[],[ชื่อ: นาย ธีรภัทร],[เลขที่อ้างอิง: 0049990],"[K, +, Accepts all banks |รับ]"
004999086098141_20250813_213350.jpeg,[],[],[],[],[ชื่อ: นาย ธีรภัทร],[เลขที่อ้างอิง: 0049990],"[K, +, Accepts all banks |รับ]"
004999086098141_20251119_161009.jpeg,[],[],[],[],[ชื่อ: นาย ธีรภัทร พ],"[เลขทีอ้างอิง: 004999, Q]","[K, Accepts all banks รับ]"
014359142508COR04950.jpeg,[24 ธ.ค.67 14:25 น],[นาย ณัฐยุว์ จิตสงวน],[ธ.กรุงศรีอยุธยา],[xxx-x-x9617-x],[014359142508C0R04950],[270.00 บาท],[0.00 บาท]
014359143117APM04609.jpeg,"[24 ธ.ค.67, 14:31 น]",[TrueMoney Wallet],"[IrueMoney Wallet, 0981702030]",[0901702050],[014359143117APM04609],[350.00 บาท],[0.00 บาท]
...,...,...,...,...,...,...,...
016008121452CPP06690.jpeg,"[8 ม.ค.69, 12:14 น.]","[นายดนุสรณ์ ใจบุญ, 6]","[6, รหัสพร้อมเพย์]",[Xxx-xxx-4937],[016008121452CPP06690],"[3,080.00 บาท]",[0.00 บาท]
016008190840130455.jpeg,"[8 ม.ค.69, 19:08 น.]",[ไลน์แมน],"[551 ro 600ะ r, 00420260108190808ZRJ]",[00420260108190808ZRJ],[016008190840130455],[215.00 บาท],[0.00 บาท]
016009173621DPM08676.jpeg,[9 ม.ค. 69 17:36 น.],[TrueMoney Wallet],"[TrueMoney Wallet, 0981702030]",[0981702030],[016009173621DPM08676],"[1,250.00 บาท]",[0.00 บาท]
016009193041631201.jpeg,[9 ม.ค. 69 19:30 น.],[ไลน์แมน],[004202601091930sZ4XR],[004202601091930sz4XR],[016009193041631201],[155.80 บาท],[0.00 บาท]
